## Compute rotational energy levels of rigid water molecule
Use the following geometrical parameters: the O--H distances are equal to 0.96 Angstrom and the H--O--H valence bond angle is 104$^\circ$

In [13]:
# Compute Cartesain coordinates of atoms, moment of itertia, and rotational constants

import numpy as np

r1 = 0.96
r2 = 0.96
theta = 104.0 * np.pi / 180.0 # angle in radians
masses = [15.99491462, 1.00782503, 1.00782503] # masses of O, H1, and H2 atoms

# Cartesian coordinates of atoms (iatom, ix) where ix=0,1,2 (=x,y,z)
xyz = np.array( [[0, 0, 0], \
                 [r1 * np.sin(theta / 2.0), 0, r1 * np.cos(theta / 2.0)], \
                 [-r2 * np.sin(theta / 2.0), 0, r2 * np.cos(theta / 2.0)]], dtype=np.float64)

# move origin to the centre of mass
com = np.dot(masses, xyz) / np.sum(masses)
xyz = xyz - com[np.newaxis, :]

# compute inertia tensor
delta = lambda a, b: 1.0 if a==b else 0.0
imom = np.zeros((3,3), dtype=np.float64)
for i in range(3):
    for j in range(3):
        imom[i,j] = -np.sum(xyz[:,i] * xyz[:,j] * masses[:]) \
                  + np.sum(np.sum(xyz[:,:]**2, axis=1) * masses[:]) * delta(i,j)

# diagonalize inertia tensor and compute rotational constants
diag, vec = np.linalg.eigh(imom)
A, B, C = 1.0 / diag
print("rotational constants in units 1/(au_mass * Angstrom^2):", A, B, C)

# Cartesian coordinates of atoms in the principal-axes frame
xyz = np.dot(xyz, vec)
print("Cartesian coordiates of atoms in the principal-axes frame:\n", xyz)

# convert rotational constants such that T = A*J_a^2 + B*J_b^2 + C*J_c^2
planck = 6.62606896e-27 # Plank constant in erg a second
avogno = 6.0221415e+23  # Avogadro constant
vellgt = 2.99792458e+10 # Speed of light constant in centimetres per second
convert_to_cm = planck * avogno * 1e+16 / (8.0 * np.pi * np.pi * vellgt)
print("conversion factor for rotational constants into units cm^-1:", convert_to_cm)
A, B, C = 1.0 / diag * convert_to_cm
print("rotational constants in units cm^-1:", A, B, C)

rotational constants in units 1/(au_mass * Angstrom^2): 1.5992039133513467 0.8669181785390033 0.5621696298954518
Cartesian coordiates of atoms in the principal-axes frame:
 [[ 0.         -0.06614561  0.        ]
 [ 0.75649032  0.52488941  0.        ]
 [-0.75649032  0.52488941  0.        ]]
conversion factor for rotational constants into units cm^-1: 16.857628229754965
rotational constants in units cm^-1: 26.958785034846276 14.614184359426858 9.47684662283647


To set up the matrix representation of Hamiltonian in symmetric-top basis we need to compute the matrix elements of $J_+^2$ and $J_-^2$ ladder operators, as well as $J_z^2$ and $J^2$. However, instead of deriving and programming the explicit expressions for the matrix elements, it is sufficient to program only functions that return the result of an operator's onto basis functions, i.e., psi2 = Jplus(psi1). The product of operators, e.g., $J_+\cdot J_+\cdot J_+ = J_+^3$, can then be expressed as consecutive calls of operator functions, e.g., psi2 = Jplus(Jplus(Jplus(psi1))). The matrix element is then simply an overlap integral.

In [14]:
# Set up functions for operators J+|psi>, J-|psi>, Jz|psi>, J^2|psi> and overlap

Jplus = lambda J, k, c=1: (J, k-1, np.sqrt(J * (J + 1) - k * (k - 1)) * c if abs(k-1) <= J else 0)
Jminus = lambda J, k, c=1: (J, k+1, np.sqrt(J * (J + 1) - k * (k + 1)) * c if abs(k+1) <= J else 0)
Jz = lambda J, k, c=1: (J, k, k * c)
Jsq = lambda J, k, c=1: (J, k, J * (J + 1) * c)

# function for overlap integral <psi'|operator psi>
overlap = lambda J1, k1, c1, J2, k2, c2: c1 * c2 * delta(J1, J2) * delta(k1, k2)

Set up the Hamiltonian using two conventions, for near-prolate ($z=a$) and near-oblate ($z=c$) top molecules, and compute eigenvalues and eigenvectors for both cases. The resulting energies for the near-prolate and near-oblate cases must be equal, however not the eigenvectors. Print assignment of each state by the $k_a$ and $k_c$ quantum numbers of the $J_z$ operator.

In [15]:
# Compute rotaitonal energies and wave functions and store them in hdf5 file

import h5py
file = h5py.File("water_states.h5", "w") # open file to store the data

Jmax = 10 # maximal value of J quantum number spanned by basis

for J in range(Jmax+1):
    # generate basis set indices (k-quanta)
    k_list = [k for k in range(-J, J+1)]

    # matrix elements of J+^2, J_^2, Jz^2, and J^2
    Jplus_mat = np.array([[overlap(*(J, k1, 1), *Jplus(*Jplus(J, k2))) for k1 in k_list] for k2 in k_list], dtype=np.float64)
    Jminus_mat = np.array([[overlap(*(J, k1, 1), *Jminus(*Jminus(J, k2))) for k1 in k_list] for k2 in k_list], dtype=np.float64)
    Jz_mat = np.array([[overlap(*(J, k1, 1), *Jz(*Jz(J, k2))) for k1 in k_list] for k2 in k_list], dtype=np.float64)
    Jsq_mat = np.array([[overlap(*(J, k1, 1), *Jsq(J, k2)) for k1 in k_list] for k2 in k_list], dtype=np.float64)
    
    # Hamiltonian for near-prolate top
    hmat_a = (Jplus_mat + Jminus_mat) * (B - C)/4.0 + Jz_mat * (2*A - B - C)/2.0 + (B + C)/2.0 * Jsq_mat
    enr_a, vec_a = np.linalg.eigh(hmat_a)
    
    # Hamiltonian for near-oblate top
    hmat_c = (Jplus_mat + Jminus_mat) * (A - B)/4.0 + Jz_mat * (2*C - A - B)/2.0 + (A + B)/2.0 * Jsq_mat
    enr_c, vec_c = np.linalg.eigh(hmat_c)

    # print energies and assignments by k_a and k_c quantum numbers
    for istate in range(len(k_list)):
        ind_a = np.argmax(vec_a[:,istate]**2)
        ind_c = np.argmax(vec_c[:,istate]**2)
        print(J, istate, " %12.6f"%enr_a[istate], " %12.6f"%enr_c[istate], "ka, kc = ", abs(k_list[ind_a]), abs(k_list[ind_c]))

    # store energies and wave functions in file
    fgroup = file.create_group("J:"+str(J))
    dset = fgroup.create_dataset('k_list', data=k_list)
    dset = fgroup.create_dataset('enr', data=enr_c)
    dset = fgroup.create_dataset('vec', data=vec_c)
        
file.close()

0 0      0.000000      0.000000 ka, kc =  0 0
1 0     24.091031     24.091031 ka, kc =  0 1
1 1     36.435632     36.435632 ka, kc =  1 1
1 2     41.572969     41.572969 ka, kc =  1 0
2 0     70.974093     70.974093 ka, kc =  0 2
2 1     79.480356     79.480356 ka, kc =  1 2
2 2     94.892369     94.892369 ka, kc =  1 1
2 3    131.926171    131.926171 ka, kc =  2 1
2 4    133.225171    133.225171 ka, kc =  2 0
3 0    138.518771    138.518771 ka, kc =  0 3
3 1    143.316754    143.316754 ka, kc =  1 3
3 2    173.927434    173.927434 ka, kc =  1 2
3 3    204.199264    204.199264 ka, kc =  2 2
3 4    210.226679    210.226679 ka, kc =  2 1
3 5    279.496300    279.496300 ka, kc =  3 1
3 6    279.709647    279.709647 ka, kc =  3 0
4 0    225.033469    225.033469 ka, kc =  0 4
4 1    227.324555    227.324555 ka, kc =  1 4
4 2    277.262887    277.262887 ka, kc =  1 3
4 3    299.536941    299.536941 ka, kc =  2 3
4 4    315.384896    315.384896 ka, kc =  2 0
4 5    377.942071    377.942071 ka

We can now run same calculations using `watie` module

In [17]:
from richmol.watie import RigidMolecule, SymtopBasis, Jxx, Jyy, Jzz

water = RigidMolecule()
water.XYZ = ("angstrom", \
             "O", 0, 0, 0, \
             "H", r1*np.sin(theta/2.0), 0, r1*np.cos(theta/2.0), \
             "H", -r2*np.sin(theta/2.0), 0, r2*np.cos(theta/2.0) )

water.frame = "pas"
Bx, By, Bz = water.B
print("Bx, By, Bz rotational constants in cm^-1:", Bx, By, Bz)

Jmax = 10

for J in range(Jmax+1):
    bas = SymtopBasis(J)

    # z=c, near-oblate
    H = Bx * Jxx(bas) + By * Jyy(bas) + Bz * Jzz(bas)
    hmat = bas.overlap(H)
    enr_c, vec = np.linalg.eigh(hmat.real)
    bas_c = bas.rotate(krot=(vec.T, enr_c))   # rotate basis to the eigenvector representation

    # z=a, near-prolate
    H = Bz * Jxx(bas) + By * Jyy(bas) + Bx * Jzz(bas)
    hmat = bas.overlap(H)
    enr_a, vec = np.linalg.eigh(hmat.real)
    bas_a = bas.rotate(krot=(vec.T, enr_a))   # rotate basis to the eigenvector representation
    
    # print states energies and assignments
    for istate in range(bas_c.nstates):
        print( J, istate, " %12.6f"%bas_a.enr[istate], " %12.6f"%bas_c.enr[istate], \
               "ka,kc =", bas_a.assign[istate][1], bas_c.assign[istate][1] )


Bx, By, Bz rotational constants in cm^-1: 26.958784982583502 14.614184327525283 9.476846602963152
0 0      0.000000      0.000000 ka,kc = 0 0
1 0     24.091031     24.091031 ka,kc = 0 1
1 1     36.435632     36.435632 ka,kc = 1 1
1 2     41.572969     41.572969 ka,kc = 1 0
2 0     70.974093     70.974093 ka,kc = 0 2
2 1     79.480356     79.480356 ka,kc = 1 2
2 2     94.892369     94.892369 ka,kc = 1 1
2 3    131.926171    131.926171 ka,kc = 2 1
2 4    133.225170    133.225170 ka,kc = 2 0
3 0    138.518771    138.518771 ka,kc = 0 3
3 1    143.316754    143.316754 ka,kc = 1 3
3 2    173.927433    173.927433 ka,kc = 1 2
3 3    204.199264    204.199264 ka,kc = 2 2
3 4    210.226679    210.226679 ka,kc = 2 1
3 5    279.496300    279.496300 ka,kc = 3 1
3 6    279.709646    279.709646 ka,kc = 3 0
4 0    225.033469    225.033469 ka,kc = 0 4
4 1    227.324555    227.324555 ka,kc = 1 4
4 2    277.262887    277.262887 ka,kc = 1 3
4 3    299.536941    299.536941 ka,kc = 2 3
4 4    315.384895    3

Remark: for some reason, rotational constants calculated explicitly above and obtained from `watie` have slightly differrent values (need to find the reason), which is reflected in the assignment of rotaitonal states for high $J$.